# Convert dataframes to excel files
This will never replace any files, but it will download dataframes if they are missing locally, and convert them into excel files if they are missing locally.

1. Inspect what dataframe files are available in the gs://entsoe bucket
2. For each dataframe file online:
   - consider if a dataframe file is available locally
   - consider if an excel file is available locally
   - if needed, optionally download the dataframe and then generate an excel file

In [10]:
import os
import pickle

import pandas as pd

bucket_name = "gs://entsoe/"
dataframe_files_dir = "dataframe_files"
excel_files_dir = "excel_files"

In [ ]:
!mkdir -p {dataframe_files_dir}
!mkdir -p {excel_files_dir}

In [16]:
def convert_dataframe_to_excel(dataframe_file_path):
    """
    Accept local files and files in a remote bucket.
    """    
    if dataframe_file_path.startswith(bucket_name):
        dataframe_file_path = dataframe_file_path.replace(bucket_name, "")
    excel_file_name = os.path.basename(dataframe_file_path).replace(".pickle", ".xlsx")
    excel_file_path = os.path.join(excel_files_dir, excel_file_name)
    
    if not os.path.isfile(dataframe_file_path):
        # NOTE: If this exit with an error code, execution will still continue,
        #       but fail later due to a missing file.
        !gsutil cp {bucket_name + dataframe_file_path} {dataframe_file_path}
    if os.path.isfile(excel_file_path):
        return
    
    with open(dataframe_file_path, 'rb') as f:
        se_df = pickle.load(f)

    with pd.ExcelWriter(excel_file_path) as excel_writer:
        se_df.tz_convert(tz=None).to_excel(excel_writer)
    
    print(f"Converted {dataframe_file_path} to {excel_file_path}.")
    return excel_file_path

In [17]:
for filename in os.listdir(dataframe_files_dir):
    if filename.endswith(".pickle"):
        excel_file = convert_dataframe_to_excel(os.path.join(dataframe_files_dir, filename))

Converted dataframe_files/fr.pickle to excel_files/fr.xlsx.
Converted dataframe_files/ch.pickle to excel_files/ch.xlsx.
Converted dataframe_files/pl.pickle to excel_files/pl.xlsx.
Converted dataframe_files/me.pickle to excel_files/me.xlsx.
Converted dataframe_files/rs.pickle to excel_files/rs.xlsx.
Converted dataframe_files/ee.pickle to excel_files/ee.xlsx.
Converted dataframe_files/hu.pickle to excel_files/hu.xlsx.
Converted dataframe_files/se.pickle to excel_files/se.xlsx.
Converted dataframe_files/gb.pickle to excel_files/gb.xlsx.
Converted dataframe_files/fi.pickle to excel_files/fi.xlsx.
Converted dataframe_files/sk.pickle to excel_files/sk.xlsx.
Converted dataframe_files/lt.pickle to excel_files/lt.xlsx.
Converted dataframe_files/gr.pickle to excel_files/gr.xlsx.
Converted dataframe_files/al.pickle to excel_files/al.xlsx.
Converted dataframe_files/de-transnetbw.pickle to excel_files/de-transnetbw.xlsx.
Converted dataframe_files/de-50hertz.pickle to excel_files/de-50hertz.xlsx.
Co

In [18]:
# sync local files to remote bucket
!gsutil rsync {excel_files_dir} {bucket_name + excel_files_dir}


both the source and destination. Your crcmod installation isn't using the
module's C extension, so checksumming will run very slowly. If this is your
first rsync since updating gsutil, this rsync can take significantly longer than
usual. For help installing the extension, please see "gsutil help crcmod".

Building synchronization state...
Starting synchronization...
Copying file://excel_files/al.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
Copying file://excel_files/at.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
Copying file://excel_files/ba.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
Copying file://excel_files/be.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
\ [4 files][ 12.4 MiB/ 12.4 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run